the data is located /Users/deepakmahtani/.kaggle/competitions/titanic

First thing to do is to import some standard packages

In [1]:
import numpy as np
import pandas as pd
import pandas_profiling
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler

Now lest read in the training data and test data

In [2]:
raw_training=pd.read_csv('/Users/deepakmahtani/.kaggle/competitions/titanic/train.csv')
raw_test=pd.read_csv('/Users/deepakmahtani/.kaggle/competitions/titanic/test.csv')

Now lets look at the columns

In [3]:
print('The column labels are',list(raw_training.columns))

The column labels are ['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']


After chatting with John the columns we are going to use are Survived, pclass, sex, age,SibSp and Parch

In [4]:
training_to_use=raw_training[['PassengerId', 'Survived', 'Pclass','Sex','Age','SibSp', 'Parch']]


test_to_use=raw_test[['PassengerId', 'Pclass','Sex','Age','SibSp', 'Parch']]

I know that the gender column is very important but we need to change this from male and female to a numeric value so lets change this in both the training and text set

In [5]:
training_to_use.replace(to_replace='male', value=0, inplace=True)
training_to_use.replace(to_replace='female', value=1, inplace=True)

test_to_use.replace(to_replace='male', value=0, inplace=True)
test_to_use.replace(to_replace='female', value=1, inplace=True)

/Users/deepakmahtani/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/deepakmahtani/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/deepakmahtani/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/dee

now lets replace the missing age values in the data with the median value. The reason we have chosen the median age and not the mean age is beacuse it is less affected by outliers

In [6]:
median_age_training=np.median(training_to_use[pd.notnull(training_to_use['Age'])]['Age'])

median_age_test=np.median(test_to_use[pd.notnull(test_to_use['Age'])]['Age'])

print("median age for the training set", median_age_training)
print("median age for the test set", median_age_test)

median age for the training set 28.0
median age for the test set 27.0


In [7]:
clean_age_train=training_to_use['Age'].fillna(median_age_training)
clean_age_test=test_to_use['Age'].fillna(median_age_test)

training_to_use['clean_age']=clean_age_train
test_to_use['clean_age']=clean_age_test

/Users/deepakmahtani/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/deepakmahtani/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


Now lets creat the two data frames to be used for the training and testing of the model

In [8]:
clean_data_training=training_to_use[['PassengerId','Survived', 'Pclass', 'Sex',
                                     'SibSp', 'Parch','clean_age']]

clean_data_test=test_to_use[['PassengerId', 'Pclass', 'Sex','SibSp', 'Parch', 'clean_age']]

Next I will initiate the regression object

In [9]:
logistic_reg=LogisticRegression()

we also need to scale the features such that they are between 0 and 1, so I will use the minmax scaler for this. and then do this on the features x

In [10]:
scaler = MinMaxScaler()

X=clean_data_training[['Pclass', 'Sex','SibSp', 'Parch','clean_age']]

X_scaled=scaler.fit_transform(X)

y=clean_data_training['Survived']

In [11]:
print(X_scaled)

[[1.         0.         0.125      0.         0.27117366]
 [0.         1.         0.125      0.         0.4722292 ]
 [1.         1.         0.         0.         0.32143755]
 ...
 [1.         1.         0.125      0.33333333 0.34656949]
 [0.         0.         0.         0.         0.32143755]
 [1.         0.         0.         0.         0.39683338]]


Now I will conduct 5-fold cross validation to see how well this model performs. The default scoring system for the random forrest is accuracy

In [12]:
accuracy = cross_val_score(logistic_reg, X_scaled, y, cv=5)

In [13]:
print('so the accuracy for this model varies from',min(accuracy), 'to', max(accuracy))

so the accuracy for this model varies from 0.7584269662921348 to 0.807909604519774


In [14]:
accuracy

array([0.79888268, 0.79329609, 0.79213483, 0.75842697, 0.8079096 ])

so now lets see how well it does on the test set, first lets fit the model on the training data

In [15]:
logistic_reg_test=LogisticRegression()
logistic_reg_test.fit(X_scaled,y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

now lets creat the training data on which to make the predictions and scale the features

In [16]:
X_test=clean_data_test[['Pclass', 'Sex','SibSp', 'Parch','clean_age']]
X_test_scaled=scaler.fit_transform(X_test)

now lets use the model to make the predictions

In [18]:
predicts=logistic_reg_test.predict(X_test)

creat a data frame to upload to kaggle to see how well it performs

In [19]:
results=clean_data_test[['PassengerId']]
results['Survived']=predicts

/Users/deepakmahtani/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [20]:
results.to_csv('log_reg.csv',index=False)

This scored 0.62679 which is not better than the raw Random forrest